In [10]:
import keras
import keras.layers as KL
import torch.utils.data as TD
from keras.models import Sequential
import numpy as np
import os
import sys
from tensorboardX import SummaryWriter
import time
from keras.utils import to_categorical

sys.path.append('..')
from utils.metrics import runningScore,get_scores
from dataset.cityscapes import cityscapes
from models.keras.semantic_segmentation import SS

from keras import backend as K
K.clear_session()
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)

In [11]:
config = SS.get_default_config()
config.model.activation='softmax'
config.training.log_dir='/home/yzbx/tmp/logs/keras'
config.dataset.resize_shape=(36,36)
config.model.input_shape=(36,36)

class_number = config.model.class_number
input_size=config.model.input_shape.copy()
if len(input_size)==2:
    input_size.append(3)
            
model=Sequential([
            KL.Conv2D(filters=class_number,
                      kernel_size=1,
                      strides=1,
                      padding='same',
                      activation='softmax',
                     input_shape=input_size)])

In [12]:
batch_size=2
train_dataset=cityscapes(config.dataset,split='train',bchw=False)
train_loader=TD.DataLoader(dataset=train_dataset,batch_size=32, shuffle=True,drop_last=False)

val_dataset=cityscapes(config.dataset,split='val',bchw=False)
val_loader=TD.DataLoader(dataset=val_dataset,batch_size=32, shuffle=True,drop_last=False)

Found 2975 image files, 8925 annotation files
Found 500 image files, 1500 annotation files


In [13]:
running_metrics = runningScore(class_number)
name='empty_net'
time_str = time.strftime("%Y-%m-%d___%H-%M-%S", time.localtime())
log_dir=os.path.join(config.training.log_dir,name,config.dataset.name,config.training.note,time_str)
checkpoint_path=os.path.join(log_dir,"{}_{}_best_model.pkl".format(name, config.dataset.name))
os.makedirs(log_dir,exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)
best_iou=0.0

In [14]:
for images,labels in train_loader:
    print(type(images),images.shape)
    print(type(labels),labels.shape)
    break

<class 'torch.Tensor'> torch.Size([32, 36, 36, 3])
<class 'torch.Tensor'> torch.Size([32, 36, 36])


In [15]:
model.compile(loss='mse',optimizer='adam',metrics=['acc'])

In [16]:
x=images.data.numpy()
np_labels=labels.data.numpy()
b,h,w=np_labels.shape
y=to_categorical(np_labels,class_number)

In [25]:
print(x.shape)
print(y.shape)
train_outputs=model.train_on_batch(x=x,y=y)
print(model.metrics_names)
print(train_outputs)
predict_outputs=model.predict_on_batch(x)
print(type(predict_outputs))
print(predict_outputs.shape)
test_outputs=model.test_on_batch(x,y)
print(type(test_outputs))
print(test_outputs)

(32, 36, 36, 3)
(32, 36, 36, 20)
['loss', 'acc']
[0.0992285, 0.0021942516]
<class 'numpy.ndarray'>
(32, 36, 36, 20)
<class 'list'>
[0.09916329, 0.0022907022]


['loss', 'acc']
